In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Orange3_colab/Orange3/Random_forest_Gradient_Boosting/2017DC1.csv')
df.shape

(20495, 30)

In [4]:
data = df.drop(['VALP_B1'], axis = 1)
target = df['VALP_B1']
data.shape

(20495, 29)

In [5]:
target.shape

(20495,)

In [6]:
# 5:5 데이터 분할~
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size = 0.5, random_state = 42
)

In [8]:
#랜덤 포레스트 모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators = 100, random_state = 0)
model = rf.fit(X_train, y_train)

#학습된 Classifier로 테스트 데이터세트를 이용해서 타깃 변수 예측값 생성
pred = model.predict(X_test)

print("Accuracy on training setL {:.5f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

Accuracy on training setL 1.00000
Accuracy on test set:0.83109


In [10]:
#정확도 100%로 과적합
#그리드 서치

rf = RandomForestClassifier(n_estimators = 100, random_state = 0)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

#StratifiedKFold의 random_state 옵션값을 0으로 고정
cross_validation = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
params = {'max_depth': range(20, 31), 'n_estimators': [100, 200]}

#GridSearchCV의 cv=cross_validation 옵션값은 StratifiedKFold의 random_state 옵션값을 적용해서 GridSearchCV를 실행할 때마다 항상 동일한 결과가 나오도록 보장
grid_rf = GridSearchCV(
    rf, param_grid = params, scoring = 'accuracy', cv = cross_validation, verbose = 1, n_jobs = -1
)
grid_rf.fit(X_train, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_rf.best_score_))
print("GridSearchCV best parameter:", (grid_rf.best_params_))

Fitting 5 folds for each of 22 candidates, totalling 110 fits
GridSearchCV max accuracy:0.82922
GridSearchCV best parameter: {'max_depth': 28, 'n_estimators': 200}
